In [1]:
import pandas as pd
from datetime import *
from dateutil import *
import warnings
import numpy as np
import os
warnings.filterwarnings('ignore')

In [5]:
def concat_excel_files(dir_path):
    # Get a list of all Excel files in the directory
    excel_files = [f for f in os.listdir(dir_path) if f.endswith('.xlsx')]

    # Initialize an empty list to store the DataFrames
    dfs = pd.DataFrame()

    # Loop through the Excel files and read them into DataFrames
    for excel_file in excel_files:
        df = pd.read_excel(os.path.join(dir_path, excel_file))
        columns_keep = ['Ítem', 'Sucursal', 'Número Suministro', 'Número Medidor',
       'Ruta / Libro', 'Dirección', 'Operador', 'Fecha Solicitud',
       'Fecha Generación', 'Fecha Ejecución', 'Fecha Pago', 'Acción Realizada',
       'Situación Encontrada', 'Lectura', 'F - T - A', 'Sticker',
       'Observaciones', 'Tipo Proceso', 'Efectividad', 'Estado Suministro',
       'Cant. Fotos', 'Ubicación Medidor', 'Prioridad']
        df = df[columns_keep]
        dfs = pd.concat([dfs,df])   

    return dfs

In [6]:
df = concat_excel_files(r'D:\BD_PROCESAR\CX')
df1 = concat_excel_files(r'D:\BD_PROCESAR\RXN')

In [7]:
df['ACCIÓN'] = df['Acción Realizada'].str.split("=",expand = True)[0]
df['ACCIÓN'] = df['ACCIÓN'].str.strip().str.upper()

df1['ACCIÓN'] = df1['Acción Realizada'].str.split("=",expand = True)[0]
df1['ACCIÓN'] = df1['ACCIÓN'].str.strip().str.upper()

df['SECTOR'] = df['Ruta / Libro'].str[:2] 
df1['SECTOR'] = df1['Ruta / Libro'].str[:2] 

df['EFECTIVIDAD'] = np.where(df['Efectividad'] == 'Efectivo','Efectivo','No efectivo')
df['EFECTIVIDAD'] = np.where(df['ACCIÓN']== 'NV','No visitado',df['EFECTIVIDAD'])
df['EFECTIVIDAD'] = np.where(df['ACCIÓN'] == 'PG','PG',df['EFECTIVIDAD'])

df1['EFECTIVIDAD'] = np.where(df1['Efectividad'] == 'Efectivo','Efectivo','No efectivo')
df1['EFECTIVIDAD'] = np.where(df1['ACCIÓN']== 'NV','No visitado',df1['EFECTIVIDAD'])
df1['EFECTIVIDAD'] = np.where(df1['ACCIÓN'] == 'PG','PG',df1['EFECTIVIDAD'])

df['Operador'] = df['Operador'].str.upper().str.strip()
df1['Operador'] = df1['Operador'].str.upper().str.strip()

df['Fecha Ejecución'] = pd.to_datetime(df['Fecha Ejecución'],format = '%d/%m/%Y %H:%M')
df1['Fecha Ejecución'] = pd.to_datetime(df1['Fecha Ejecución'],format = '%d/%m/%Y %H:%M')
df['HORA EJEC'] = df['Fecha Ejecución'].dt.time
df1['HORA EJEC'] = df1['Fecha Ejecución'].dt.time

df['Fecha Generación'] = pd.to_datetime(df['Fecha Generación'],format = '%d/%m/%Y %H:%M')
df1['Fecha Generación'] = pd.to_datetime(df1['Fecha Generación'],format = '%d/%m/%Y %H:%M')

df['FECHA EJEC'] = df['Fecha Ejecución'].dt.strftime('%d/%m/%Y')
df['FECHA EJEC'] = pd.to_datetime(df['FECHA EJEC'],format = '%d/%m/%Y')

df1['FECHA EJEC'] = df1['Fecha Ejecución'].dt.strftime('%d/%m/%Y')
df1['FECHA EJEC'] = pd.to_datetime(df1['FECHA EJEC'],format = '%d/%m/%Y')

df['FECHA GEN'] = df['Fecha Generación'].dt.strftime('%d/%m/%Y')
df['FECHA GEN'] = pd.to_datetime(df['FECHA GEN'],format = '%d/%m/%Y')

df1['FECHA GEN'] = df1['Fecha Generación'].dt.strftime('%d/%m/%Y')
df1['FECHA GEN'] = pd.to_datetime(df1['FECHA GEN'],format = '%d/%m/%Y')

df.dropna(subset = 'FECHA EJEC',inplace=True)
df1.dropna(subset = 'FECHA EJEC',inplace=True)
df.reset_index(drop = True,inplace = True)
df1.reset_index(drop = True,inplace = True)
df['TIPO TRABAJO'] = 'Corte'
df1['TIPO TRABAJO'] = 'Reconexión'
df_of = pd.concat([df,df1])
df_of.reset_index(drop = True,inplace = True)
df_of.drop_duplicates(subset=['Número Suministro','Fecha Generación','Fecha Ejecución','TIPO TRABAJO','ACCIÓN'],inplace = True)
lista_fechas = df_of['FECHA EJEC'].unique()
df_efectivo = df_of[df_of['EFECTIVIDAD'].isin(['Efectivo','No efectivo'])]

In [ ]:
df_final = pd.DataFrame()
for i in lista_fechas:
    df_temp = df_efectivo[df_efectivo['FECHA EJEC'] == i]
    lista_operarios = df_efectivo['Operador'].unique()
    for j in lista_operarios:
        df_temp_2 = df_temp[df_temp['Operador'] == j]
        df_temp_2.sort_values(by = 'HORA EJEC',inplace=True)
        df_temp_2['HORA EJEC'] = pd.to_datetime(df_temp_2['HORA EJEC'],format = '%H:%M:%S')
        df_temp_2['Diferencia'] = df_temp_2['HORA EJEC'].diff(periods=1)
        temp_group = df_temp_2.groupby(['Operador','FECHA EJEC'],as_index = False).agg({'Diferencia':['min','max','mean']})
        new_columns = ['Operador', 'FECHA EJEC', 'min', 'max', 'mean']
        temp_group.columns = new_columns
        df_final = pd.concat([df_final,temp_group])
        df_final.reset_index(drop = True,inplace=True)
df_final['min'] =df_final['min'].dt.total_seconds()/60
df_final['max'] =df_final['max'].dt.total_seconds()/60
df_final['mean'] =df_final['mean'].dt.total_seconds()/60

In [9]:
now_date = datetime.today().strftime('%Y%m%d_%H%M')

In [10]:
df_final.to_excel(rf'D:\BD_PROCESAR\PROCESADO\PROCESADO_{now_date}.xlsx',index = False)
df_of.to_excel(rf'D:\BD_PROCESAR\PROCESADO\BD_CONCATENADA_{now_date}.xlsx',index = False)